# 1. Importación de paquetes

In [6]:
#Import packages
from modules import preprocess
import pandas as pd

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

## Paso 1: Lectura del *dataset* y obtención de datos

In [7]:
df = preprocess.load_csv_data('../dataset/Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [8]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 2: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [9]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [10]:
#Process "Room_Occupancy_Count"
preprocess.recode_dataset_output(df)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

## Paso 3: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.

In [11]:
# Filter Date and Time columns
preprocess.remove_time_columns(df)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 4: División del *dataset* en *training* y *test*

In [12]:
train_df, test_df = preprocess.split_dataset(df)

## Paso 5: División del *dataset* de entrenamiento en *features* y *output*

In [13]:
# Step 2: split training into features and output
X = preprocess.get_features(train_df)
y = preprocess.get_output(train_df)

# 2. Entrenamiento y seguimiento de la huella de carbono

### Codecarbon:

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from codecarbon import EmissionsTracker

In [15]:
lg_pipeline = Pipeline([("scaler", StandardScaler()), ("logistic_regression", LogisticRegression())])
tracker = EmissionsTracker()

tracker.start()
lg_pipeline.fit(X, y)
emissions: float = tracker.stop()
print(f"Emissions: {emissions} kg")

[codecarbon INFO @ 17:57:34] [setup] RAM Tracking...
[codecarbon INFO @ 17:57:34] [setup] GPU Tracking...
[codecarbon INFO @ 17:57:34] No GPU found.
[codecarbon INFO @ 17:57:34] [setup] CPU Tracking...
[codecarbon WARNING @ 17:57:34] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:57:36] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:57:36] >>> Tracker's metadata:
[codecarbon INFO @ 17:57:36]   Platform system: Linux-5.15.0-47-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 17:57:36]   Python version: 3.10.4
[codecarbon INFO @ 17:57:36]   Available RAM : 1.930 GB
[codecarbon INFO @ 17:57:36]   CPU count: 1
[codecarbon INFO @ 17:57:37]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:57:37]   GPU count: None
[codecarbon INFO @ 17:57:37]   GPU model: None
[codecarbon INFO @ 17:57:42] Energy consumed for RAM : 0.000000 kWh. RAM Power : 0.7236471176147461 W
[codecarbon I

Emissions: 1.5511533908960018e-06 kg


In [16]:
# Show results
codecarbon_emissions = pd.read_csv("emissions.csv",sep=",")
codecarbon_emissions

,timestamp,project_name,run_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,cpu_energy,...,python_version,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud
0,2022-10-05T17:57:42,codecarbon,81a6f627-4d3e-43b3-b3ae-c30c7e525607,0.885733,0.000002,0.001751,32.5,0.0,0.723647,0.000008,...,3.10.4,1,Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz,NaN,NaN,-3.8661,40.3192,1.929726,machine,N
